In [ ]:
import cluster
import acquire
import prepare
import summarize

In [ ]:
df = acquire.wrangle_zillow()

In [ ]:
df.info()

In [ ]:
summarize.nulls_by_col(df)

    Getting rid of some ID columns, and the date one. 
    
    LAT-LONG stays because the split_data function cuts it off.

In [ ]:
#df = df.drop(['transaction_date','property_land_use_code', 'id', 'parcel_id'], axis=1)
df = df.drop(['transaction_date','property_land_use_code', 'id', 'parcel_id','zipcode_id','fips_code', 'logerror', 'abs_logerror'], axis=1)

In [ ]:
df = df.drop(['latitude','longitude'], axis=1)

# FOR THE MOMENT. NO TEST TRAIN SPLIT FOR MAKING CLUSTERS. THATS FOR LATER

In [ ]:
df_standard, df2, std_object = cluster.standardize_train_test(df, df)

In [ ]:
df_standard.info()

## Remove some outliers, maybe

In [ ]:
# prepare.remove_upper_outliers(df_standard., df_standard)

## It might be helpful

In [ ]:
cluster.list_inertia_scores(df_standard)

In [ ]:
three_cluster_df = cluster.make_clusters(df_standard, n_clusters=3)

In [ ]:
two_cluster_df = cluster.make_clusters(df_standard, n_clusters=2)

In [ ]:
three_cluster_df['cluster_labels'].value_counts()

In [ ]:
two_cluster_df['cluster_labels'].value_counts()

In [ ]:
53683/len(two_cluster_df)

In [ ]:
cluster.show_clusters_on_map(two_cluster_df)

In [ ]:
cluster_labels = two_cluster_df.cluster_labels

In [ ]:
two_cluster_df.drop(['latitude','longitude','cluster_labels'],axis=1,inplace=True)

In [ ]:
scaler, unscaled_df = cluster.my_inv_transform(std_object, two_cluster_df)

In [ ]:
unscaled_df.head()

In [ ]:
unscaled_df['cluster_labels'] = cluster_labels

In [ ]:
unscaled_df.groupby('cluster_labels').mean()

In [ ]:
cluster.show_clusters_on_map(unscaled_df)

In [ ]:
unscaled_df.abs_logerror.mean()

In [ ]:
unscaled_df.logerror.mean()

In [ ]:
unscaled_df.groupby('cluster_labels').logerror.mean()

In [ ]:
unscaled_df.groupby('cluster_labels').logerror.std()

In [ ]:
unscaled_df.logerror.std()

In [ ]:
unscaled_df.groupby('cluster_labels').total_value.mean()

In [ ]:
unscaled_df.groupby('cluster_labels').tax_amount.mean()

In [ ]:
sns.boxplot(data=unscaled_df, x='cluster_labels', y='logerror')
plt.yscale('log')